## Testing Julia 

In [ ]:
using LinearAlgebra

# create a simple array of 2d points
a = [(1.,2.), (3.,4.), (5.,6.)]
a = [("a", 1.1), ("b", 2.2), ("c", 3.3)]

norm_a = norm.(a)
cos_a = cos.(a)  # this will fail because cos expects a number, not a tuple

println("type of a = ", typeof(a))
println("type of norm_a = ", typeof(norm_a))
println("norm_a = ", norm_a)


In [ ]:
methods(cos)

In [ ]:
using LinearAlgebra

# now create an array of 2d vectors
a = [ [1.,2.], [3.,4.], [5.,6.]]

norm_a = norm.(a)
cos_a = cos.(a)

println("type of a = ", typeof(a))
println("type of norm_a = ", typeof(norm_a))
println("type of cos_a = ", typeof(cos_a))
println("norm_a = ", norm_a)
println("cos_a = ", cos_a)

In [ ]:
using LinearAlgebra

#create a 2d matrix
a = [1. 2.; 3. 4.; 5. 6.]

norm_a = norm.(a)
cos_a = cos.(a)

println("type of a = ", typeof(a))
println("type of norm_a = ", typeof(norm_a))
println("type of cos_a = ", typeof(cos_a))
println("norm_a = ", norm_a)
println("cos_a = ", cos_a)

In [ ]:
using LinearAlgebra

ma, mb = [1. 2. 3.; 4. 5. 6.], [7. 8. 9.; 10. 11. 12.]
va, vb = [[1.,2.,3.], [4.,5.,6.]], [[7.,8.,9.], [10.,11.,12.]]
a, b = [1., 2., 3.], [7., 8., 9.]

In [ ]:
a

In [ ]:
va

In [ ]:
map(x -> a-x, va)

In [ ]:
(a,) .- va

In [ ]:
a × b

In [ ]:
va .× vb

In [ ]:
ma[:] × mb